In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/data/2_merged_da_data.csv')
df = df.sort_values(['LAT','LON'],ascending=[False, True])
df.head()

,LAT,LON,LAT_25,LON_25,LAT_5,LON_5,knutson_precip,updated_precip,knutson_temp,updated_temp,...,precip_da_cat,temp_da_cat,temp_da_binary,precip_da_binary,n_da_impacts,max_da_cat,study_addition,total_study_prop,area,is_land
10224,88.75,-178.75,88.75,-178.75,87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10225,88.75,-176.25,88.75,-176.25,87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10226,88.75,-173.75,88.75,-173.75,87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10227,88.75,-171.25,88.75,-171.25,87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10228,88.75,-168.75,88.75,-168.75,87.5,-167.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False


In [3]:
df.columns

Index(['LAT', 'LON', 'LAT_25', 'LON_25', 'LAT_5', 'LON_5', 'knutson_precip',
       'updated_precip', 'knutson_temp', 'updated_temp', 'population',
       'diff_temp', 'diff_precip', 'index', 'all_study_prop',
       'all_study_prop_upper', 'all_study_prop_lower', 'WA', 'temp_study_prop',
       'temp_study_prop_upper', 'temp_study_prop_lower', 'precip_study_prop',
       'precip_study_prop_upper', 'precip_study_prop_lower', 'precip_da_cat',
       'temp_da_cat', 'temp_da_binary', 'precip_da_binary', 'n_da_impacts',
       'max_da_cat', 'study_addition', 'total_study_prop', 'area', 'is_land'],
      dtype='object')

In [4]:
len(df.LAT.unique()),len(df.LON.unique())

(72, 144)

In [5]:
df[['LAT','LON','precip_da_cat']].to_csv('data/da_data.csv',index=False)

In [6]:
d = {
    "width": 144,
    "height": 72,
    "scale": [2.5, -2.5],
    "translate": [180, 90],
    "values": list(df.temp_da_cat)
}
import json
with open('data/da_data.json', 'w') as fp:
    json.dump(d, fp)

In [7]:
df.columns

Index(['LAT', 'LON', 'LAT_25', 'LON_25', 'LAT_5', 'LON_5', 'knutson_precip',
       'updated_precip', 'knutson_temp', 'updated_temp', 'population',
       'diff_temp', 'diff_precip', 'index', 'all_study_prop',
       'all_study_prop_upper', 'all_study_prop_lower', 'WA', 'temp_study_prop',
       'temp_study_prop_upper', 'temp_study_prop_lower', 'precip_study_prop',
       'precip_study_prop_upper', 'precip_study_prop_lower', 'precip_da_cat',
       'temp_da_cat', 'temp_da_binary', 'precip_da_binary', 'n_da_impacts',
       'max_da_cat', 'study_addition', 'total_study_prop', 'area', 'is_land'],
      dtype='object')

In [8]:
df = df.rename(columns={
    "updated_temp":"Temperature",
    "updated_precip":"Precipitation",
    "temp_study_prop": "Weighted studies on temperature",
    "all_study_prop": "Weighted studies"
})
d = {
    "type": "FeatureCollection",
    "features": []
}
df.loc[pd.isna(df['precip_da_cat']),"precip_da_cat"] = 5
df.loc[pd.isna(df['temp_da_cat']),"temp_da_cat"] = 5
cols = [
    "Temperature","Precipitation",
    "Weighted studies on temperature","Weighted studies",
    "LAT","LON"
]
cols = df.columns

for i, row in df.fillna(5).iterrows():
    rdict = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [row.LON-1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT-1.25],                
            ]
        }, 
        "properties": {
            
        }
    }
    for c in cols:
        if "LATasdf" in c or "LONasdf" in c:
            continue
        rdict[c] = row[c]
        rdict["properties"][c] = row[c]
    
    d['features'].append(rdict)
    
with open('data/da_data.json', 'w') as fp:
    json.dump(d, fp)

In [9]:
rdict

{'type': 'Feature',
 'geometry': {'type': 'LineString',
  'coordinates': [[177.5, -90.0],
   [177.5, -87.5],
   [180.0, -87.5],
   [180.0, -90.0],
   [177.5, -90.0]]},
 'properties': {'LAT': -88.75,
  'LON': 178.75,
  'LAT_25': -88.75,
  'LON_25': 178.75,
  'LAT_5': -87.5,
  'LON_5': 177.5,
  'knutson_precip': 5.0,
  'Precipitation': 5.0,
  'knutson_temp': 5.0,
  'Temperature': 5.0,
  'population': 0.0,
  'diff_temp': 5.0,
  'diff_precip': 5.0,
  'index': 143,
  'Weighted studies': 0.0,
  'all_study_prop_upper': 0.0,
  'all_study_prop_lower': 0.0,
  'WA': 5.0,
  'Weighted studies on temperature': 0.0,
  'temp_study_prop_upper': 0.0,
  'temp_study_prop_lower': 0.0,
  'precip_study_prop': 0.0,
  'precip_study_prop_upper': 0.0,
  'precip_study_prop_lower': 0.0,
  'precip_da_cat': 0.0,
  'temp_da_cat': 0.0,
  'temp_da_binary': 0,
  'precip_da_binary': 0,
  'n_da_impacts': 0,
  'max_da_cat': 0.0,
  'study_addition': 'add',
  'total_study_prop': 0.0,
  'area': 1685.6540146695665,
  'is_land'

In [10]:
c = ["#ffed6f","#b2182b","#ef8a62","#fddbc7","#f7f7f7","#d1e5f0","#67a9cf","#2166ac","#33a02c"]

In [26]:
long_df = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/data/2b_merged_da_data_impacts.csv')
long_df = long_df.sort_values(['LAT','LON'],ascending=[False, True])
#long_df['impact'] = [x.split()[0] for x in long_df['impact']]
long_df.head()

,index,LAT,LON,weight,driver,impact
10224,10224,88.75,-178.75,0.462597,all,all
20592,10224,88.75,-178.75,0.075289,all,Coastal and marine Ecosystems
30960,10224,88.75,-178.75,0.001570,all,Human and managed
41328,10224,88.75,-178.75,0.224276,all,"Mountains, snow and ice"
51696,10224,88.75,-178.75,0.015664,all,"Rivers, lakes, and soil moisture"


In [27]:
# df = df.rename(columns={
#     "updated_temp":"Temperature",
#     "updated_precip":"Precipitation",
#     "temp_study_prop": "Weighted studies on temperature",
#     "all_study_prop": "Weighted studies"
# })
d = {
    "type": "FeatureCollection",
    "features": []
}
cols = [x for x in long_df.columns if x!="index"]
for i, row in long_df.fillna(5).iterrows():
    rdict = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [row.LON-1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT-1.25],                
            ]
        }, 
        "properties": {
            
        }
    }
    for c in cols:
        if "LATasdf" in c or "LONasdf" in c:
            continue
        rdict[c] = row[c]
        rdict["properties"][c] = row[c]
    
    d['features'].append(rdict)
    
with open('data/ws_data.json', 'w') as fp:
    json.dump(d, fp)